Results for:

In [12]:
AGENT_NAME = r'Models\ATLA\PPO agent 100 alts over 1000+200 2-3-21.zip'
#DATASET_NAME = '19 feature 300 episodes results\schema.json'
DATASET_NAME = 'citylearn_challenge_2022_phase_2'
SAVE_DIR = 'PPO agent 100 alts over 1000+200 2-3-21 results' + '/'
ATK_NAME = 'dynamic_untargeted_acg'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion import AutoConjugateGradient as ACG

import pandas as pd
import numpy as np
import json

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
try: #try to load CityLearn schema
    schema = DataSet.get_schema(DATASET_NAME)
except: #load saved schema otherwise
    with open(DATASET_NAME, 'r') as file:
        schema = json.load(file)

Define RL agent

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}",
                 print_system_info=True)
print('Model loaded from storage')

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.23.5
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0



c:\Users\Broda-Milian\anaconda3\envs\CityLearnART\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from 'c:\\Users\\Broda-Milian\\anaconda3\\envs\\CityLearnART\\lib\\site-packages\\cloudpickle\\cloudpickle.py'>
  warnings.warn(
c:\Users\Broda-Milian\anaconda3\envs\CityLearnART\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from 'c:\\Users\\Broda-Milian\\anaconda3\\envs\\CityLearnART\\lib\\site-packages\\cloudpickle\\cloudpickle.py'>
  warnings.warn(


Model loaded from storage


In [5]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [6]:
cols = env.observation_names

In [7]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features

In [8]:
init = 50
iter = int(500/init)
ACG_parameters = dict(
    loss_type='difference_logits_ratio', 
    batch_size=1,
    nb_random_init=init, #5, lower values speed crafting
    max_iter=iter, #iterations per restart
    norm='inf', #->l2 ->l1 most restrictive 
)
eps_list = [0.01, 0.03, 0.05, 0.07]

In [9]:
%%time
#%%capture
#attack = utils.define_attack(agent, ACG)
kpi, obs, adv_obs, eps = utils.eval_untargeted_dynamic_distortion_attack(agent,
                                                        env,
                                                        ART_atk=ACG,
                                                        ART_atk_kwargs=ACG_parameters,
                                                        eps_candidates=eps_list,
                                                        time_steps=None,
                                                        mask=observation_masks)

100%|█████████▉| 8758/8759 [3:19:29<00:01,  1.37s/it]  


The Adversarial success rate is: 0.9899531910035392
The average distance between optinmal and adversarial actions is: 2.5317958671081238
CPU times: total: 3h 19min 17s
Wall time: 3h 19min 31s


calculate asr from eps

KPIs <=1 are win, since we aren't worse off using a DRL controller in the worst case attack

In [10]:
kpi

cost_function
annual_peak_average                      1.009986
carbon_emissions_total                   0.906728
cost_total                               0.799590
daily_one_minus_load_factor_average      1.024541
daily_peak_average                       0.954933
electricity_consumption_total            0.924883
monthly_one_minus_load_factor_average    0.989035
ramping_average                          1.213821
zero_net_energy                          1.105068
Name: District, dtype: float64

In [13]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/KPIs.csv updated


In [14]:
df_obs = pd.DataFrame(obs)
df_obs.columns = cols
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_obs.csv')

In [15]:
df_obs = pd.DataFrame(adv_obs)
df_obs.columns = cols
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_adv_obs.csv')

In [16]:
eps = pd.Series(eps)
eps_savename = SAVE_DIR+'epsilons.csv'
try:
    df_eps = pd.read_csv(eps_savename)
    df_eps[ATK_NAME] = eps.values
    df_eps.to_csv(eps_savename)
    print(f'{eps_savename} updated')
except:
    eps.name = ATK_NAME
    eps.to_csv(eps_savename)
    print(f'{eps_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/epsilons.csv created


In [17]:
asr = eps.notna().mean()
asr_savename = SAVE_DIR+'ASRs.csv'
try:
    df_asrs = pd.read_csv(asr_savename)
    df_asrs[ATK_NAME] = asr
    df_asrs.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    asr = pd.Series([asr])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print(f'{asr_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/ASRs.csv created
